In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def SSH_chain(N,v,w):
    # Creating first half of the SSH chain
    T0 = np.eye(N)
    Th = np.zeros([N, N])
    for i in range(N-1):
        Th[i, i+1] = 1
    t0 = np.array([[0, v], [v, 0]])
    th = np.array([[0, 0], [w, 0]])
    psi = np.kron(T0, t0) + np.kron(Th,th) + np.transpose(np.conj(np.kron(Th, th)))
    return psi

In [4]:
def mol_solSSH(N, v, w, e0, dlt, T1):
    # Number of pair of solitons;
    size = np.random.randint(1, (N-40)//80 + 1)
    nn1 = 2 * np.sort(np.random.randint(40//2, (N-40)//2+1, size))

    length_nn1 = len(nn1)
    i = 1
    while i < length_nn1:
        # check if the difference between the current element and the previous element is greater than 40
        if abs(nn1[i] - nn1[i-1]) < 40:
            # if true, delete the current element
            nn1 = np.delete(nn1, i)
            # update the loop counter and array length
            length_nn1 -= 1
        else:
            # if false, move to the next element
            i += i

    if len(nn1) % 2 == 1: # Odd number of elements
        # Determine the index of the middle element
        mid_index = len(nn1) // 2
        # Remove the middle element
        nn = np.concatenate((nn1[:mid_index], nn1[mid_index+1:], [N]))
    else: # Even number of elements
        nn = np.concatenate((nn1, [N]))

    NN = N + len(nn) + 1
    nn[-1] = N
    solSSH = np.zeros((NN,NN))

    soliton = np.diag(((w-v)*np.tanh((np.linspace(0,40,41)-20)/7)+(v+w))/2,1) \
         + np.conj(np.diag(((w-v)*np.tanh((np.linspace(0,40,41)-20)/7)+(v+w))/2,1))
    ansoliton = np.diag((-(w-v)*np.tanh((np.linspace(0,40,41)-20)/7)+(v+w))/2,1) \
         + np.conj(np.diag((-(w-v)*np.tanh((np.linspace(0,40,41)-20)/7)+(v+w))/2,1))

    # First SSH chain up the the first defect
    solSSH[:nn[0]-20, :nn[0]-20] = SSH_chain((nn[0]-20)//2, v, w)

    for i in range(0, len(nn)-1, 2):
        solSSH[nn[i]-20:nn[i]+20+1, nn[i]-20:nn[i]+20+1] = soliton
        solSSH[nn[i]+20+1:nn[i+1]-20, nn[i]+20+1:nn[i+1]-20] = SSH_chain((nn[i+1]-nn[i]-40)//2, w, v)
        solSSH[nn[i+1]-20:nn[i+1]+20+1, nn[i+1]-20:nn[i+1]+20+1] = ansoliton
        solSSH[nn[i+1]+20+1:nn[i+2]-20, nn[i+1]+20+1:nn[i+2]-20] = SSH_chain((nn[i+2]-nn[i+1]-40)//2, v, w)

    solSSH[N-20-1:N, N-20-1:N] = SSH_chain(11, v, w)
    nn = nn[:-1]
    e00 = e0 - dlt/2 + dlt*np.random.rand(len(nn)+1)
    solSSH[0,N] = -T1; solSSH[N,0] = -T1
    solSSH[1,N] = -T1/3; solSSH[N,1] = -T1/3
    solSSH[N+1,N+1] = e00[0]

    for i in range(0, leng(nn)):
        solSSH[nn[i],N+1+i] = -T1; solSSH[N+1+i,nn[i]] = -T1
        solSSH[nn[i]+1,N+1+i] = -T1/3; solSSH[N+1+i,nn[i]+1] = -T1/3
        solSSH[nn[i]-1,N+1+i] = -T1/3; solSSH[N+1+i,nn[i]-1] = -T1/3
        solSSH[N+1+i,N+1+i] = e00(i+1)

In [1]:
import numpy as np
import matplotlib.pyplot as plt

N = 6080
v = 10
w = 15
T1 = 0.1
e0 = 0.1
dlt = 0.05

import numpy as np

def solSSH(N, v, w, e0, dlt, T1, xi):
    # SSH Chain parameters
    size = np.random.randint(1, (N-xi*6)//(xi*10) + 1)
    nn1 = 2 * np.sort(np.random.randint(xi*3, (N-xi*6)//2 + 1, size))
    
    if len(nn1) % 2 == 1: # Odd number of elements
        # Remove the first element
        nn = nn1[1:]
    else: # Even number of elements
        nn = nn1
    NN = N + len(nn) + 1
    # solSSH = np.zeros((NN, NN))
    # SSH chain
    ss1 = np.zeros(N-1)
    for i in range(1, len(nn), 2):
        ss1 += (w-v)*(np.tanh((np.linspace(1,N-1,N-1)-nn[i-1])/xi) - np.tanh((np.linspace(1,N-1,N-1)-nn[i])/xi))/2
    ss1[1::2] = -ss1[1::2]
    tp = (w-v)*np.ones(N-1)/2; tp[0::2] = -tp[0::2]
    ss = (w+v)/2 + tp + ss1; ss = np.append(ss, np.zeros(len(nn)+1))

    solSSH = np.diag(ss, k=1) + np.diag(np.conj(ss), k=-1)
        
    # Adparticle energies (random distribution of energy)
    e00 = e0 - dlt/2 + dlt*np.random.rand(len(nn)+1)
    solSSH[0, N] = -T1; solSSH[N, 0] = -T1
    solSSH[1, N] = -T1/3; solSSH[N, 1] = -T1/3
    solSSH[N, N] = e00[0]

    for i in range(len(nn)):
        solSSH[nn[i], N+i+1] = -T1; solSSH[N+i+1, nn[i]] = -T1
        solSSH[nn[i]+1, N+i+1] = -T1; solSSH[N+i+1, nn[i]+1] = -T1
        # solSSH[nn[i]-1, N+i+1] = -T1/3; solSSH[N+i+1, nn[i]-1] = -T1/3;
        solSSH[N+i+1, N+i+1] = e00[i+1]
    nn = np.insert(nn, 0, 1)
    
    return solSSH, nn



In [6]:

[H sol] = 
idx = D.argsort()[::-1]   
D = D[idx]
V = V[:,idx]
# Plot the diagonal elements
plt.plot(V[:,200-5])
plt.xlabel('Index')
plt.ylabel('Value')
plt.title('Diagonal Elements of Matrix')
plt.show()

NameError: name 'D' is not defined